In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.8 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer,AutoModelForSequenceClassification

In [ ]:
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# nli_model = AutoModelForSequenceClassification.from_pretrained("ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli")
# tokenizer = AutoTokenizer.from_pretrained("ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli")

In [ ]:
print(device)

cuda


In [ ]:
def compute_asr(df,logic):
  nli_model = AutoModelForSequenceClassification.from_pretrained("ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli")
  tokenizer = AutoTokenizer.from_pretrained("ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli")
  nli_model.to(device)
  label_dict = {0:"entailment",1:"neutral",2:"contradiction"}
  if logic != 'Overall':
    df = df[df['propositional_logic_rule'] == logic]
  premises = df['premise'].values
  hypotheses = df['hypothesis'].values
  logics = df['propositional_logic_rule'].values
  sum = 0

  for premise,hypothesis,a in zip(premises,hypotheses,logics):
    x = tokenizer.encode(premise, hypothesis, return_tensors='pt')
    logits = nli_model(x.to(device))[0]
    probs = logits.softmax(dim=1)
    probs = probs.cpu().data.numpy()
    label = np.argmax(probs)
    if a != "Negate Hypothesis":
      if label_dict[label] != "entailment":
        sum += 1
    else:
      if label_dict[label] != "contradiction":
        sum += 1
  return [sum,len(premises) - sum,sum/len(premises)]

In [ ]:
df = pd.read_csv("/content/Stage_2_BART_Dataset.csv")
print(df.shape)

(3137, 8)


In [ ]:
def modus_tollens(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2):
    while positive_sentence1[-1]=='.':
        positive_sentence1 = positive_sentence1[:-1]
    while negative_sentence1[-1]=='.':
        negative_sentence1 = negative_sentence1[:-1]
    while positive_sentence2[-1]=='.':
        positive_sentence2 = positive_sentence2[:-1]
    while negative_sentence2[-1]=='.':
        negative_sentence2 = negative_sentence2[:-1]
    sentence1 = positive_sentence1[0].lower() + positive_sentence1[1:]
    sentence2 = positive_sentence2[0].lower() + positive_sentence2[1:]
    premise =  "If " + sentence1 + ", then " + sentence2 + ". " + negative_sentence2+"."
    hypothesis = negative_sentence1+"."
    return premise,hypothesis

In [ ]:
def constructive_dilemma(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2, positive_sentence3, negative_sentence3, positive_sentence4, negative_sentence4):
    while positive_sentence1[-1]=='.':
        positive_sentence1 = positive_sentence1[:-1]
    while negative_sentence1[-1]=='.':
        negative_sentence1 = negative_sentence1[:-1]
    while positive_sentence2[-1]=='.':
        positive_sentence2 = positive_sentence2[:-1]
    while negative_sentence2[-1]=='.':
        negative_sentence2 = negative_sentence2[:-1]
    while positive_sentence3[-1]=='.':
        positive_sentence3 = positive_sentence3[:-1]
    while negative_sentence3[-1]=='.':
        negative_sentence3 = negative_sentence3[:-1]
    while positive_sentence4[-1]=='.':
        positive_sentence4 = positive_sentence4[:-1]
    while negative_sentence4[-1]=='.':
        negative_sentence4 = negative_sentence4[:-1]
    sentence1 = positive_sentence1[0].lower() + positive_sentence1[1:]
    sentence2 = positive_sentence2[0].lower() + positive_sentence2[1:]
    sentence3 = positive_sentence3[0].lower() + positive_sentence3[1:]
    sentence4 = positive_sentence4[0].lower() + positive_sentence4[1:]
    premise = "If " + sentence1 +", then " + sentence2+". And if "+ sentence3 +", then " + sentence4+". But either "+ sentence1 + " or " + sentence3+"."
    hypothesis = "Either "+sentence2 +" or " + sentence4+"."
    return premise,hypothesis

In [ ]:
def destructive_dilemma(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2, positive_sentence3, negative_sentence3, positive_sentence4, negative_sentence4):
    while positive_sentence1[-1]=='.':
        positive_sentence1 = positive_sentence1[:-1]
    while negative_sentence1[-1]=='.':
        negative_sentence1 = negative_sentence1[:-1]
    while positive_sentence2[-1]=='.':
        positive_sentence2 = positive_sentence2[:-1]
    while negative_sentence2[-1]=='.':
        negative_sentence2 = negative_sentence2[:-1]
    while positive_sentence3[-1]=='.':
        positive_sentence3 = positive_sentence3[:-1]
    while negative_sentence3[-1]=='.':
        negative_sentence3 = negative_sentence3[:-1]
    while positive_sentence4[-1]=='.':
        positive_sentence4 = positive_sentence4[:-1]
    while negative_sentence4[-1]=='.':
        negative_sentence4 = negative_sentence4[:-1]
    sentence1 = positive_sentence1[0].lower() + positive_sentence1[1:]
    sentence2 = positive_sentence2[0].lower() + positive_sentence2[1:]
    sentence3 = positive_sentence3[0].lower() + positive_sentence3[1:]
    sentence4 = positive_sentence4[0].lower() + positive_sentence4[1:]
    sentence1_n = negative_sentence1[0].lower() + negative_sentence1[1:]
    sentence2_n = negative_sentence2[0].lower() + negative_sentence2[1:]
    sentence3_n = negative_sentence3[0].lower() + negative_sentence3[1:]
    sentence4_n = negative_sentence4[0].lower() + negative_sentence4[1:]
    premise = "If " + sentence1 +", then " + sentence2+". And if "+ sentence3 +", then " + sentence4+". But either "+ sentence2_n + " or " + sentence4_n+"."
    hypothesis = "Either "+ sentence1_n  +" or " + sentence3_n +"."
    return premise,hypothesis

In [ ]:
def bidirectional_dilemma(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2, positive_sentence3, negative_sentence3, positive_sentence4, negative_sentence4):
    while positive_sentence1[-1]=='.':
        positive_sentence1 = positive_sentence1[:-1]
    while negative_sentence1[-1]=='.':
        negative_sentence1 = negative_sentence1[:-1]
    while positive_sentence2[-1]=='.':
        positive_sentence2 = positive_sentence2[:-1]
    while negative_sentence2[-1]=='.':
        negative_sentence2 = negative_sentence2[:-1]
    while positive_sentence3[-1]=='.':
        positive_sentence3 = positive_sentence3[:-1]
    while negative_sentence3[-1]=='.':
        negative_sentence3 = negative_sentence3[:-1]
    while positive_sentence4[-1]=='.':
        positive_sentence4 = positive_sentence4[:-1]
    while negative_sentence4[-1]=='.':
        negative_sentence4 = negative_sentence4[:-1]

    sentence1 = positive_sentence1[0].lower() + positive_sentence1[1:]
    sentence2 = positive_sentence2[0].lower() + positive_sentence2[1:]
    sentence3 = positive_sentence3[0].lower() + positive_sentence3[1:]
    sentence4 = positive_sentence4[0].lower() + positive_sentence4[1:]
    sentence1_n = negative_sentence1[0].lower() + negative_sentence1[1:]
    sentence2_n = negative_sentence2[0].lower() + negative_sentence2[1:]
    sentence3_n = negative_sentence3[0].lower() + negative_sentence3[1:]
    sentence4_n = negative_sentence4[0].lower() + negative_sentence4[1:]
    premise = "If " + sentence1 +", then " + sentence2+". And if "+ sentence3 +", then " + sentence4+". But either "+ sentence1 + " or " + sentence4_n+"."
    hypothesis = "Either "+sentence2 +" or " + sentence3_n+"."
    return premise,hypothesis

In [ ]:
def transportation_1(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2):
    while positive_sentence1[-1]=='.':
        positive_sentence1 = positive_sentence1[:-1]
    while negative_sentence1[-1]=='.':
        negative_sentence1 = negative_sentence1[:-1]
    while positive_sentence2[-1]=='.':
        positive_sentence2 = positive_sentence2[:-1]
    while negative_sentence2[-1]=='.':
        negative_sentence2 = negative_sentence2[:-1]

    sentence1 = positive_sentence1[0].lower() + positive_sentence1[1:]
    sentence2 = positive_sentence2[0].lower() + positive_sentence2[1:]
    sentence1_n = negative_sentence1[0].lower() + negative_sentence1[1:]
    sentence2_n = negative_sentence2[0].lower() + negative_sentence2[1:]
    premise =  "If " + sentence1 + ", then " + sentence2 + "."
    hypothesis = "If " + sentence2_n + ", then " + sentence1_n + "."
    return premise,hypothesis

In [ ]:
def transportation_2(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2):
    premise,hypothesis = transportation_1(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2)
    return hypothesis,premise

In [ ]:
def material_implication_1(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2):
    while positive_sentence1[-1]=='.':
        positive_sentence1 = positive_sentence1[:-1]
    while negative_sentence1[-1]=='.':
        negative_sentence1 = negative_sentence1[:-1]
    while positive_sentence2[-1]=='.':
        positive_sentence2 = positive_sentence2[:-1]
    while negative_sentence2[-1]=='.':
        negative_sentence2 = negative_sentence2[:-1]

    sentence1 = positive_sentence1[0].lower() + positive_sentence1[1:]
    sentence2 = positive_sentence2[0].lower() + positive_sentence2[1:]
    sentence1_n = negative_sentence1[0].lower() + negative_sentence1[1:]
    sentence2_n = negative_sentence2[0].lower() + negative_sentence2[1:]
    premise =  "If " + sentence1 + ", then " + sentence2 + "."
    hypothesis = "Either " + sentence1_n + " or " + sentence2 + "."
    return premise,hypothesis

In [ ]:
def material_implication_2(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2):
    premise,hypothesis = material_implication_1(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2)
    return hypothesis,premise

In [ ]:
def negate_hypthesis(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2):
    while positive_sentence1[-1]=='.':
        positive_sentence1 = positive_sentence1[:-1]
    while negative_sentence1[-1]=='.':
        negative_sentence1 = negative_sentence1[:-1]
    while positive_sentence2[-1]=='.':
        positive_sentence2 = positive_sentence2[:-1]
    while negative_sentence2[-1]=='.':
        negative_sentence2 = negative_sentence2[:-1]
    premise = positive_sentence1 +"."
    hypothesis = negative_sentence2 + "."
    return premise,hypothesis

In [ ]:
df_result = pd.DataFrame(columns=['pairID1','pairID2', 'premise', 'hypothesis','propositional_logic_rule'])
df_result

,pairID1,pairID2,premise,hypothesis,propositional_logic_rule


In [ ]:
#Expressions involving 1 row
for i,row in df.iterrows():
    positive_sentence1 = row["sentence1"]
    negative_sentence1 = row["sentence1_n"]
    positive_sentence2 = row["sentence2"]
    negative_sentence2 = row["sentence2_n"]
    pair_id = row["pairID"]

    modus_tollens_premise, modus_tollens_hypothesis = modus_tollens(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2)
    row = [pair_id, '', modus_tollens_premise, modus_tollens_hypothesis,'Modus Tollens']
    df_result.loc[len(df_result)] = row

    transportation_1_premise, transportation_1_hypothesis = transportation_1(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2)
    row = [pair_id, '', transportation_1_premise, transportation_1_hypothesis,'Transportation 1']
    df_result.loc[len(df_result)] = row

    transportation_2_premise, transportation_2_hypothesis = transportation_2(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2)
    row = [pair_id, '', transportation_2_premise, transportation_2_hypothesis,'Transportation 2']
    df_result.loc[len(df_result)] = row

    material_implication_1_premise, material_implication_1_hypothesis = material_implication_1(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2)
    row = [pair_id, '', material_implication_1_premise, material_implication_1_hypothesis,'Material Implication 1']
    df_result.loc[len(df_result)] = row

    material_implication_2_premise, material_implication_2_hypothesis = material_implication_2(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2)
    row = [pair_id, '', material_implication_2_premise, material_implication_2_hypothesis,'Material Implication 2']
    df_result.loc[len(df_result)] = row

    negate_hypthesis_premise, negate_hypthesis_hypothesis = negate_hypthesis(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2)
    row = [pair_id, '', negate_hypthesis_premise, negate_hypthesis_hypothesis,'Negate Hypothesis']
    df_result.loc[len(df_result)] = row

In [ ]:
positive_sentence1 = df.iloc[0]["sentence1"]
negative_sentence1 = df.iloc[0]["sentence1_n"]
positive_sentence2 = df.iloc[0]["sentence2"]
negative_sentence2 = df.iloc[0]["sentence2_n"]
pair_id_1 = df.iloc[0]["pairID"]

positive_sentence3 = ""
negative_sentence3 = ""
positive_sentence4 = ""
negative_sentence4 = ""
pair_id_2 = ""
for i,row in df.iterrows():
    #Set first 2 sentences
    if i==0:
        continue
    positive_sentence3 = row["sentence1"]
    negative_sentence3 = row["sentence1_n"]
    positive_sentence4 = row["sentence2"]
    negative_sentence4 = row["sentence2_n"]
    pair_id_2 = row["pairID"]

    constructive_dilemma_premise, constructive_dilemma_hypothesis = constructive_dilemma(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2, positive_sentence3, negative_sentence3, positive_sentence4, negative_sentence4)
    row = [pair_id_1, pair_id_2, constructive_dilemma_premise, constructive_dilemma_hypothesis,'Constructive Dilemma']
    df_result.loc[len(df_result)] = row

    destructive_dilemma_premise, destructive_dilemma_hypothesis = destructive_dilemma(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2, positive_sentence3, negative_sentence3, positive_sentence4, negative_sentence4)
    row = [pair_id_1, pair_id_2, destructive_dilemma_premise, destructive_dilemma_hypothesis,'Destructive Dilemma']
    df_result.loc[len(df_result)] = row

    bidirectional_dilemma_premise, bidirectional_dilemma_hypothesis = bidirectional_dilemma(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2, positive_sentence3, negative_sentence3, positive_sentence4, negative_sentence4)
    row = [pair_id_1, pair_id_2, bidirectional_dilemma_premise, bidirectional_dilemma_hypothesis,'Bidirectional Dilemma']
    df_result.loc[len(df_result)] = row

    positive_sentence1 = positive_sentence3
    negative_sentence1 = negative_sentence3
    positive_sentence2 = positive_sentence4
    negative_sentence2 = negative_sentence4
    pair_id_1 = pair_id_2

In [ ]:
positive_sentence3 = df.iloc[0]["sentence1"]
negative_sentence3 = df.iloc[0]["sentence1_n"]
positive_sentence4 = df.iloc[0]["sentence2"]
negative_sentence4 = df.iloc[0]["sentence2_n"]
pair_id_2 = df.iloc[0]["pairID"]

constructive_dilemma_premise, constructive_dilemma_hypothesis = constructive_dilemma(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2, positive_sentence3, negative_sentence3, positive_sentence4, negative_sentence4)
row = [pair_id_1, pair_id_2, constructive_dilemma_premise, constructive_dilemma_hypothesis,'Constructive Dilemma']
df_result.loc[len(df_result)] = row

destructive_dilemma_premise, destructive_dilemma_hypothesis = destructive_dilemma(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2, positive_sentence3, negative_sentence3, positive_sentence4, negative_sentence4)
row = [pair_id_1, pair_id_2, destructive_dilemma_premise, destructive_dilemma_hypothesis,'Destructive Dilemma']
df_result.loc[len(df_result)] = row

bidirectional_dilemma_premise, bidirectional_dilemma_hypothesis = bidirectional_dilemma(positive_sentence1, negative_sentence1, positive_sentence2, negative_sentence2, positive_sentence3, negative_sentence3, positive_sentence4, negative_sentence4)
row = [pair_id_1, pair_id_2, bidirectional_dilemma_premise, bidirectional_dilemma_hypothesis,'Bidirectional Dilemma']
df_result.loc[len(df_result)] = row

In [ ]:
print(df_result['propositional_logic_rule'].value_counts())

Modus Tollens             3137
Transportation 1          3137
Transportation 2          3137
Material Implication 1    3137
Material Implication 2    3137
Negate Hypothesis         3137
Constructive Dilemma      3137
Destructive Dilemma       3137
Bidirectional Dilemma     3137
Name: propositional_logic_rule, dtype: int64


In [ ]:
logic_dict = {}
logic_dict['Modus Tollens'] = compute_asr(df_result,'Modus Tollens')
logic_dict['Constructive Dilemma'] = compute_asr(df_result,'Constructive Dilemma')
logic_dict['Destructive Dilemma'] = compute_asr(df_result,'Destructive Dilemma')
logic_dict['Bidirectional Dilemma'] = compute_asr(df_result,'Bidirectional Dilemma')
logic_dict['Transportation 1'] = compute_asr(df_result,'Transportation 1')
logic_dict['Transportation 2'] = compute_asr(df_result,'Transportation 2')
logic_dict['Material Implication 1'] = compute_asr(df_result,'Material Implication 1')
logic_dict['Material Implication 2'] = compute_asr(df_result,'Material Implication 2')
logic_dict['Negate Hypothesis'] = compute_asr(df_result,'Negate Hypothesis')
logic_dict['Overall'] = compute_asr(df_result,'Overall')

In [ ]:
for key in logic_dict.keys():
    logic_dict[key][2] = round(logic_dict[key][2]* 100,2)
    print("Logic: %s ASR: %.2f"%(key,logic_dict[key][2]))

Logic: Modus Tollens ASR: 45.14
Logic: Constructive Dilemma ASR: 0.13
Logic: Destructive Dilemma ASR: 24.96
Logic: Bidirectional Dilemma ASR: 22.47
Logic: Transportation 1 ASR: 99.11
Logic: Transportation 2 ASR: 94.49
Logic: Material Implication 1 ASR: 94.71
Logic: Material Implication 2 ASR: 45.90
Logic: Negate Hypothesis ASR: 2.01
Logic: Overall ASR: 47.66


In [ ]:
for key in logic_dict.keys():
    print("Logic: %s Hits: %d Misses: %d"%(key,logic_dict[key][0],logic_dict[key][1]))

Logic: Modus Tollens Hits: 1416 Misses: 1721
Logic: Constructive Dilemma Hits: 4 Misses: 3133
Logic: Destructive Dilemma Hits: 783 Misses: 2354
Logic: Bidirectional Dilemma Hits: 705 Misses: 2432
Logic: Transportation 1 Hits: 3109 Misses: 28
Logic: Transportation 2 Hits: 2964 Misses: 173
Logic: Material Implication 1 Hits: 2971 Misses: 166
Logic: Material Implication 2 Hits: 1440 Misses: 1697
Logic: Negate Hypothesis Hits: 63 Misses: 3074
Logic: Overall Hits: 13455 Misses: 14778
